In [102]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import commonfunctions
from skimage import io, color
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import minmax_scale
from commonfunctions import show_images, showHist
from sklearn import svm
from sklearn import metrics

In [111]:
def get_paragraph(gray_img, bin_img):

    height, width = gray_img.shape

    contours, hierarchy = cv.findContours(bin_img, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    threshold_width = 1500


    up, down, left, right = 0, height - 1, 0, width - 1

    for cnt in contours:
        x, y, w, h = cv.boundingRect(cnt)

        if w < threshold_width:
            continue

        if y < height / 2:
            if y > up:
                up = y
        else:
            down = y

    th = 0
    bin_img = bin_img[up:down + 1, left:right + 1]
    gray_img = gray_img[up:down + 1, left:right + 1]
    # Apply erosion to remove noise and dots.
    kernel = np.ones((3, 3), np.uint8)
    bin_img = cv.erode(bin_img, kernel, iterations=3)
    pts = np.nonzero(bin_img)
    x_min, y_min, x_max, y_max = min(pts[0]), min(pts[1]), max(pts[0]), max(pts[1])
    bin_img = bin_img[x_min-th:x_max+th, y_min-th:y_max+th]
    gray_img = gray_img[x_min-th:x_max+th, y_min-th:y_max+th]
    # Return the handwritten paragraph
    return gray_img, bin_img

def preprocessing(gray_img):
    gray_img = cv.GaussianBlur(gray_img, (5, 5), 0)
    ## (2) threshold
    thresh, bin_img = cv.threshold(gray_img, 0, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)
    gray_img, bin_img = get_paragraph(gray_img, bin_img)
    thresh, bin_img = cv.threshold(gray_img, 0, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)
#     plt.imshow(bin_img)
    hist = cv.reduce(bin_img,1, cv.REDUCE_AVG).reshape(-1)
#     for i in range(len(hist)):
#         print(i, hist[i])
    th = 2
    H,W = bin_img.shape[:2]
    uppers = []
    lowers = []
    if hist[0] > th:
        uppers.append(0)
    
     
    for y in range(H-1):
        if hist[y]<=th and hist[y+1]>th:
            uppers.append(y)
     
    for y in range(H-1):
        if hist[y]>th and hist[y+1]<=th:
            lowers.append(y)
            
    if hist[len(hist)-1] > th:
        lowers.append(len(hist)-1)

    img = cv.cvtColor(gray_img, cv.COLOR_GRAY2BGR)
    
    lines = []
    temp_uppers = uppers.copy()
    temp_lowers = lowers.copy()
    for i in range(len(uppers)):
        if lowers[i] - uppers[i] > 50:
            lines.append(img[uppers[i]:lowers[i], :])
        else:
            temp_uppers.remove(uppers[i])
            temp_lowers.remove(lowers[i])
    print(temp_uppers)
    print(temp_lowers)
    count = 1
    for l in lines:
        #cv.imwrite("line" + str(count) + ".png", l)
        count+=1
    
    return lines

In [112]:
def LBP_feature_extraction(lines, features, labels, label):
    R = 3
    P = 8
    featureVectors = []
    for i in range(len(lines)):
        #rgb to grat scale
        grayImage = cv.cvtColor(lines[i], cv.COLOR_BGR2GRAY)
        #calc PBL
        LBPImage = local_binary_pattern(grayImage, P, R)
        #change format for histogram function
        LBPImage = np.uint8(LBPImage)     
        #calculate the histogram
        LBPHist = cv.calcHist([LBPImage],[0],None,[256],[0,256])
        #normalize the histogram 0-1
        normalizedHist = minmax_scale(LBPHist)
        #normalizedHist = LBPHist/np.mean(LBPHist)
        #print(normalizedHist[:,0])
        featureVectors.append(normalizedHist)
        features.append(normalizedHist[:,0])
        labels.append(label)
        #plot histogram
        #plt.hist(normalizedHist, bins=256)
        #plt.show()
    return featureVectors

In [113]:
def get_features(picsPath,features,labels,ids):
    for i in range(len(picsPath)):
        img = cv.imread(picsPath[i])
        gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        lines = preprocessing(gray_img)
        LBP_feature_extraction(lines, features, labels, ids[i])

In [114]:
def train_using_svm(features,labels):
    clf = svm.SVC(kernel='linear') # Linear Kernel
    clf.fit(features, labels)
    return clf

In [115]:
def testing(clf,traingPicsPath):
    ids = []
    for i in range(len(traingPicsPath)):
       ids.append(i+1) 
    trainF = []
    trainLabels = []
    get_features(traingPicsPath,trainF,trainLabels,ids)
    trainF = np.array(trainF)
    trainLabels = np.array(trainLabels)
    y_pred = clf.predict(trainF)
    print("Accuracy:",metrics.accuracy_score(trainLabels, y_pred))

In [110]:
features = []
labels = []
picsPath = ['Writerss/009/a01-020.png','Writerss/009/a01-026.png','Writerss/010/a01-020x.png',
            'Writerss/010/a01-030x.png','Writerss/011/a01-038x.png','Writerss/011/a01-043x.png']
ids = [1,1,2,2,3,3]
get_features(picsPath,features,labels,ids)
#print(type(features))
#print(labels.shape)
features = np.array(features)
labels = np.array(labels)
#print(features.shape)
clf = train_using_svm(features,labels)
traingPicsPath = ['Writerss/009/a01-020.png']
testing(clf,traingPicsPath)


[0, 239, 420, 424, 610, 777, 965, 1138, 1320, 1496, 1679]
[8, 325, 422, 512, 695, 905, 1081, 1267, 1449, 1586, 1753]


<ipython-input-98-566e62e50581>:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  lines.remove(line)


[0, 169, 351, 518, 700, 878, 1055, 1236, 1417, 1595, 1778]
[8, 281, 441, 621, 793, 976, 1168, 1343, 1508, 1689, 1819]
[0, 221, 404, 584, 760, 939, 1123, 1295, 1471, 1653]
[4, 325, 502, 678, 865, 1041, 1224, 1400, 1585, 1736]
[0, 164, 346, 521, 699, 886, 1061, 1234, 1414, 1592]
[4, 261, 427, 634, 811, 975, 1166, 1344, 1521, 1704]
[0, 102, 480, 664, 1003, 1202]
[5, 466, 650, 991, 1170, 1661]
[0, 137, 337, 877, 1237, 1417]
[4, 297, 857, 1209, 1361, 1534]
[0, 239, 420, 424, 610, 777, 965, 1138, 1320, 1496, 1679]
[8, 325, 422, 512, 695, 905, 1081, 1267, 1449, 1586, 1753]


<ipython-input-98-566e62e50581>:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  lines.remove(line)


Accuracy: 1.0


In [ ]:
img = cv.imread('Writerss/010/a01-020x.png')
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
show_images([gray_img])
lines = preprocessing(gray_img)
print("number of lines before",len(lines))
for line in lines:
    if line.shape[0] < 10:
        lines.remove(line)
print("number of lines after",len(lines))
show_images(lines)
#LBP_feature_extraction([lines[1]])
features = LBP_feature_extraction(lines)
len(features)